INSTALLATION
--

In [ ]:
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install tensorflow==2.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


IMPORT LIB.
--

In [ ]:
import sys
import gym
import math
import time
import pylab
import cmath
import random
import itertools
import threading
import tensorflow
import scipy as sp
import numpy as np
import pandas as pd
from gym import Env
import scipy.io as sc
import tensorflow as tf
from sys import version
from absl import logging
from numpy import ndarray
from scipy import special
from gym.utils import seeding
from scipy.constants import *
from rl.agents import DQNAgent
from scipy.special import erfinv
from scipy.integrate import quad
from scipy.linalg import toeplitz
from numba import jit, njit, prange
from gym.spaces import Discrete, Box
from collections import deque, Counter
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from mpl_toolkits.mplot3d import Axes3D
from gym import Env, error, spaces, utils
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input

# Set of times noew roman font.
from matplotlib import cm
from matplotlib import colors
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

dtype = np.float32

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

# import tensorflow as tf
# tf.compat.v1.disable_v2_behavior()
##############################################################################

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py:546: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class IteratorBase(collections.Iterator, trackable.Trackable,
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:106: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class DatasetV2(collections.Iterable, tracking_base.Trackable,


AI-GYM CARTPOLE
--

In [ ]:
"""
Classic cart-pole system implemented by Rich Sutton et al.
Copied from http://incompleteideas.net/sutton/book/code/pole.c
permalink: https://perma.cc/C9ZM-652R
--------
Change action type from Discrete to Box.
"""

import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np


class CartPoleEnv(gym.Env):
    """
    Description:
        A pole is attached by an un-actuated joint to a cart, which moves along
        a frictionless track. The pendulum starts upright, and the goal is to
        prevent it from falling over by increasing and reducing the cart's
        velocity.
    Source:
        This environment corresponds to the version of the cart-pole problem
        described by Barto, Sutton, and Anderson
    Observation:
        Type: Box(4)
        Num   Observation               Min             Max
        0     Cart Position             -4.8            4.8
        1     Cart Velocity             -Inf            Inf
        2     Pole Angle                -24 deg         24 deg
        3     Pole Velocity At Tip      -Inf            Inf
    Actions:
        Type: Box(1)
        Num   Action                    Min             Max
        0     Input force               -1              1
    Reward:
        Reward is 1 for every step taken, including the termination step
    Starting State:
        All observations are assigned a uniform random value in [-0.05..0.05]
    Episode Termination:
        Pole Angle is more than 12 degrees.
        Cart Position is more than 2.4 (center of the cart reaches the edge of
        the display).
        Episode length is greater than 200.
        Solved Requirements:
        Considered solved when the average reward is greater than or equal to
        195.0 over 100 consecutive trials.
    """

    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 50
    }

    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5  # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 10.0
        self.tau = 0.02  # seconds between state updates
        self.kinematics_integrator = 'euler'

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 2.4

        # Angle limit set to 2 * theta_threshold_radians so failing observation
        # is still within bounds.
        high = np.array([self.x_threshold * 2,
                         np.finfo(np.float32).max,
                         self.theta_threshold_radians * 2,
                         np.finfo(np.float32).max], dtype=np.float32)

        self.action_space = Discrete(2)
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)

        self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        err_msg = "%r (%s) invalid" % (action, type(action))
        assert self.action_space.contains(action), err_msg

        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag * action
        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (force + self.polemass_length * theta_dot ** 2 * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (self.length * (4.0 / 3.0 - self.masspole * costheta ** 2 / self.total_mass))
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == 'euler':
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        done = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            # Pole just fell!
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            if self.steps_beyond_done == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned done = True. You "
                    "should always call 'reset()' once you receive 'done = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_done += 1
            reward = 0.0

        return np.array(self.state), reward, done, {}

    def reset(self):
        self.state = self.np_random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return np.array(self.state)

CARTPOLE ENV TEST BEFORE TRAINING
--

In [ ]:
env = CartPoleEnv()

episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:12.0
Episode:2 Score:13.0
Episode:3 Score:13.0
Episode:4 Score:11.0
Episode:5 Score:17.0
Episode:6 Score:10.0
Episode:7 Score:11.0
Episode:8 Score:12.0
Episode:9 Score:11.0
Episode:10 Score:10.0


DQN MODEL
--

In [ ]:
# =============================================================================
# DEEP Q NETWORK (DQN)
# =============================================================================

# THIS IS DQN AGENT FOR THE CUSTOM ENV.
# IT USES NEURAL NETWORK TO APPROXIMATE Q FUNCTION, AND REPLAY MEMORY & TAGET Q NETWORK
class DQNAgent:
    def __init__(self, state_size, action_size, test_mode):

        # ARE YOU LOADING A PREVIOUS TRAINED DQN MODEL?
        # self.load_model = False
        self.load_model = test_mode

        # GET SIZE OF STATE AND ACTION
        self.state_size = state_size
        self.action_size = action_size

        # THESE ARE HYPER-PARAMERTERS FOR THE DQN
        self.discount_factor = 0.99
        self.learning_rate = 1e-3
        self.epsilon_decay = 0.999
        self.train_start = 1000
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.batch_size = 64

        # CREATE REPLAY MEMORY USING DEQUE
        self.memory = deque(maxlen=50000)

        # CREATE MAIN MODEL AND TARGET MODEL
        self.model = self.build_model()
        self.target_model = self.build_model()

        # INITIALIZE TARGET MODEL
        self.update_target_model()

        # GET TRAINED MY MODEL
        if self.load_model:
            self.model.load_weights(".cartpole.h5")

    # APPROXIMATE Q FUNCTION USING NEURAL ENTWORK, STATE IS INOUT AND Q VALUE OF EACH ACTION IS OUTPUT OF NETWORK
    def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(24, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(self.action_size, activation='linear', kernel_initializer='he_uniform'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate), metrics=['accuracy'])
        return model

    # AFTER SOME THIME INTERVAL UPDATE THE TARGET MODEL TO BE SAME WITH MODEL
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())


    # GET ACTION FROM MODEL USING EPSILON-GREEDY POLICY
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(state)
            return np.argmax(q_value[0])

    def predict_power(self, state):
      power = env.action_values[self.get_action(state)]
      return power


    # SAVE SAMPLE <STATE, ACTION, REWARD, NEXT STATE> TO THE REPLAY MEMORY
    def append_sample(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    # PICK SAMPLES RANDOMLY FORM REPLAY MEOMORY (WITH BATCH)
    def train_model(self):
        if len(self.memory) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.memory))
        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.state_size))
        action, reward, done = [], [], []

        for i in range(self.batch_size):
            update_input[i] = mini_batch[i][0]
            action.append(mini_batch[i][1])
            reward.append(mini_batch[i][2])
            update_target[i] = mini_batch[i][3]
            done.append(mini_batch[i][4])

        target = self.model.predict(update_input)
        target_val = self.target_model.predict(update_target)

        for i in range(self.batch_size):
            # Q LEARNING; GET MAXIMUM Q VALUE AT NEXT STAE FROM TRAGET MODEL
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.discount_factor * (np.amax(target_val[i]))

        # MAKE MINI-BATCH WHICH INCLUDES TARGET Q VALUE AND PREDICETED Q VALUE
        # where the verbose = 0, without loss values
        # where the verbos = 1, with loss values
        self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)

Train
--

In [ ]:
EPISODES = 1000

if __name__ == "__main__":
    env = CartPoleEnv()

    # get size of state and action from environment
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    print('observation {}'.format(env.observation_space.shape))
    print('state_size {}'.format(state_size))
    print('action_size {}'.format(action_size))

    agent = DQNAgent(state_size, action_size, test_mode = False)

    scores, episodes = [], []

    for e in range(EPISODES):
        done = False
        score = 0
        state = env.reset()
        state = np.reshape(state, [1, state_size])

        while not done:
            # get action for the current state and go one step in environment
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            # if an action make the episode end, then gives penalty of -100
            reward = reward if not done or score == 499 else -100

            # save the sample <s, a, r, s'> to the replay memory
            agent.append_sample(state, action, reward, next_state, done)
            # every time step do the training
            agent.train_model()
            score += reward
            state = next_state

            if done:
                # every episode update the target model to be same with model
                agent.update_target_model()

                # every episode, plot the play time
                score = score if score == 500 else score + 100
                scores.append(score)
                episodes.append(e)
                print("episode:", e, "  score:", score, "  memory length:", len(agent.memory), "  epsilon:", agent.epsilon)

                # if the mean of scores of last 10 episode is bigger than 490 stop training
                if np.mean(scores[-min(10, len(scores)):]) > 490:
                    sys.exit()

        # save the model
        if e % 50 == 0:
            agent.model.save_weights(".cartpole.h5", overwrite=True)

Test
--

In [ ]:
# custom evaluation function without tensorflow and keras
def DRL_evaluate(env, agent):
  eval_reward = []
  for i in range(5):
    obs = env.reset()
    episode_reward = 0
    while True:
      action = agent.get_action(obs)
      next_state, reward, done, info = env.step(action)
      episode_reward += reward
      if done:
        break
    eval_reward.append(episode_reward)
  return np.mean(eval_reward)

del agent.model
del agent
del env

env = CartPoleEnv()
agent = DQNAgent(state_size, action_size, test_mode = True)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n

res = DRL_evaluate(env, agent)
print('acheivable objective function value {}'.format(res))

acheivable objective function value 14.0
